In [1]:
import pandas as pd
import numpy as np
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
import os
import json
from pathlib import Path

# define groq key
groq_key = (
    "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"  # os.environ['groqkey']
)

In [2]:
from extraction import llmAgent
from query import DataLoader
from main import main

c:\Users\grays\anaconda3\envs\capstone\Lib\site-packages\pydantic\_internal\_fields.py:161: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
c:\Users\grays\OneDrive\Documents\GitHub\DataSci210_MedicationDeprescriber\src\extraction.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")
c:\Users\grays\anaconda3\envs\capstone\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.

In [3]:
# base_path = Path(
#     "/Users/yarg/Library/CloudStorage/OneDrive-Personal/Documents/GitHub/DataSci210_MedicationDeprescriber/Data"
# )
data_path = Path(
    r"C:\Users\grays\OneDrive\Documents\GitHub\DataSci210_MedicationDeprescriber\Data"
)

In [4]:
encounter_key="D6253A5CE371EA"

In [5]:
main(groq_key=groq_key, data_path=data_path)

Recommendation: 
continue

Reasoning: 
The patient has a documented history of bleeding gastrointestinal (GI) ulcer, which is supported by their primary diagnosis of gastrointestinal hemorrhage or bleeding, and their past medical history of GI bleeding. The patient's symptoms, such as melena and anemia, are also consistent with upper GI symptoms. However, there is no specific information provided about other conditions such as mild to moderate esophagitis, GERD, peptic ulcer disease, Barrett's esophagus, or severe esophagitis.

Label df: 
              key                                             reason  \
0  D6253A5CE371EA  example with gastrointestinal bleed should con...   

  recommendation  
0       continue  


In [8]:
llm_agent = llmAgent(groq_key=groq_key, data_path=data_path)

In [9]:
diagnosis_json = llm_agent.extract_diagnosis(encounter_key=encounter_key)
diagnosis_json

{'Mild to moderate esophagitis': False,
 'GERD': False,
 'Peptic Ulcer Disease': False,
 'Upper GI symptoms': True,
 'ICU Stress Ulcer Prophylaxis': False,
 "Barrett's Esophagus": False,
 'Chronic NSAID use with bleeding risk': False,
 'Severe esophagitis': False,
 'Documented history of bleeding GI ulcer': True,
 'Reasoning': 'The patient has a documented diagnosis of gastrointestinal bleeding (GIB), which indicates that they have experienced bleeding in the gastrointestinal tract. This suggests that they may have upper GI symptoms and a documented history of bleeding GI ulcer. However, there is no specific information about the other conditions listed.'}

In [10]:
encounter_json = llm_agent.extract_encounter_info(encounter_key=encounter_key)
encounter_json

{'Mild to moderate esophagitis': False,
 'GERD': False,
 'Peptic Ulcer Disease': False,
 'Upper GI symptoms': False,
 'ICU Stress Ulcer Prophylaxis': False,
 "Barrett's Esophagus": False,
 'Chronic NSAID use with bleeding risk': False,
 'Severe esophagitis': False,
 'Documented history of bleeding GI ulcer': True,
 'Reasoning': "The patient's primary diagnosis is 'Gastrointestinal hemorrhage, unspecified', which suggests that the patient has experienced bleeding in the gastrointestinal tract. However, the specific cause or underlying condition is not specified in the provided information. Therefore, we can only confirm the presence of a documented history of bleeding GI ulcer based on the primary diagnosis."}

In [11]:
notes_json = llm_agent.extract_notes(encounter_key=encounter_key)
notes_json

{'Mild to moderate esophagitis': False,
 'GERD': False,
 'Peptic Ulcer Disease': False,
 'Upper GI symptoms': True,
 'ICU Stress Ulcer Prophylaxis': False,
 "Barrett's Esophagus": False,
 'Chronic NSAID use with bleeding risk': False,
 'Severe esophagitis': False,
 'Documented history of bleeding GI ulcer': True,
 'Reasoning': "The patient has a history of AVMs (arteriovenous malformations) and has undergone APC (argon plasma coagulation) procedures to treat them. The patient also has a history of slow ooze in the past, which suggests a history of bleeding GI ulcer. Additionally, the patient has been experiencing melena and anemia, which are symptoms of upper GI bleeding. However, there is no mention of esophagitis, GERD, peptic ulcer disease, Barrett's esophagus, or chronic NSAID use in the patient's notes."}

In [25]:
type(notes_json)

dict

In [23]:
system = "You are a knowledgeable medical provider who specializes in medication management."
human = "{text}"
prompt = ChatPromptTemplate.from_messages(
    [("system", system), ("human", human)]
)

chain = prompt | llm_agent.llm
response = chain.invoke(
    {
        "text": f"""Based on the following json files, please provide a single explanation of the reasoning given by the 'Reasoning' key. Summarize given equal 
        weight to each. Do not add any additional information, only summarize what is given.
        {diagnosis_json}
        {encounter_json}
        {notes_json}"""
    }
)

In [24]:
response.pretty_print()

================================== Ai Message ==================================

The patient has a documented history of bleeding gastrointestinal (GI) ulcer, as indicated by their primary diagnosis of gastrointestinal hemorrhage or bleeding, and symptoms such as melena and anemia. The patient's medical history, including the presence of arteriovenous malformations (AVMs) and slow ooze, also supports this diagnosis. However, there is no specific information about other conditions such as esophagitis, GERD, peptic ulcer disease, Barrett's esophagus, or chronic NSAID use.


In [20]:
response.content

"Based on the provided JSON files, the reasoning behind the 'Reasoning' key can be summarized as follows:\n\nThe patient has a documented history of gastrointestinal bleeding (GIB) or gastrointestinal hemorrhage, which is the primary indication for a documented history of bleeding GI ulcer. This is supported by the patient's primary diagnosis, medical history, and symptoms such as melena and anemia. However, the specific cause or underlying condition of the bleeding is not specified in the provided information, and there is no mention of other conditions such as esophagitis, GERD, peptic ulcer disease, Barrett's esophagus, or chronic NSAID use. The patient's symptoms and medical history suggest the presence of upper GI symptoms, but this is not consistently confirmed across all three JSON files.\n\nIn terms of medication management, this information suggests that the patient may require medications to prevent future gastrointestinal bleeding, such as proton pump inhibitors (PPIs) or hi

In [12]:
final_dict = {}
for key in diagnosis_json.keys():
    #print(key)
    if not key == "Reasoning":
        diagnosis_bool = diagnosis_json[key]
        encounter_bool = encounter_json[key]
        notes_bool = notes_json[key]
        
        final_dict[key] = diagnosis_bool or encounter_bool or notes_bool
    
final_dict

{'Mild to moderate esophagitis': False,
 'GERD': False,
 'Peptic Ulcer Disease': False,
 'Upper GI symptoms': True,
 'ICU Stress Ulcer Prophylaxis': False,
 "Barrett's Esophagus": False,
 'Chronic NSAID use with bleeding risk': False,
 'Severe esophagitis': False,
 'Documented history of bleeding GI ulcer': True}

In [9]:
# DL = DataLoader(data_path=base_path)

In [10]:
# DL.get_label(encounter_key=encounter_key)

In [11]:
# DL.get_diagnosis_data(encounter_key=encounter_key)

In [12]:
# DL.get_encounter_data(encounter_key=encounter_key)